# Abstract 

Digitization, as one of the key outcomes of technological growth, has led to profound changes in entertainment, and therefore in the world of cinema, as well as in many other areas. As a result, the distribution and broadcasting strategy that Netflix brought to the market turned into an amazing success story in a very short period of time.

Netflix's strategy is based on the idea that consumers can access the platform's entire content catalog for a monthly price. In addition, Netflix only broadcasts its films on the web, with no theatrical or limited distribution. The approach, which is vastly different from the classic idea of ​​the Hollywood studio system, has led to significant advances for audiences, directors and studios in various ways. In this way, we can confidently say that streaming services, such as Netflix, are influencing the film industry in terms of how we access movies, what material we consume and how movies are made.

Every day, platforms such as Netflix and Amazon Prime gain more users thanks to competitive prices compared to movie theaters, and recommendation algorithms. The latter play an important role in the dissemination of romantic comedies and thrillers, obtaining some success thanks to the data of millions of users who use them. This dominant position places Internet platforms in a strong position in terms of film content. In the future, that authority could be key in determining what constitutes a "well-made film".

The impact of Internet streaming services on filmmakers has been one of the most important transformations in the world of cinema in recent years. The promise of a more open environment for filmmakers than other large studios has attracted numerous directors to the platforms, with huge ramifications in the world of cinema. Furthermore, the fact that these services have less stringent standards than cinemas makes them attractive to producers. Another important aspect concerns independent directors. Since the 1980s, when Hollywood became the hub of cinema and blockbuster films began to dominate theaters, it has been difficult for independent directors to reach large audiences. Cinemas often prefer high-budget movies as they can make a much larger profit from them. As a result, independent films have few opportunities outside of film festivals to date. However, with internet streaming services becoming a major role in the world of cinema, independent filmmakers now have the opportunity to reach a wider audience.

The purpose of this notebook is to investigate, through data, how streaming platforms have changed film production. is the world of production really fairer? How much power does the user of these platforms have?

# Data gatering
We start from two existing datasets:
* [Netflix](https://www.kaggle.com/datasets/shivamb/netflix-shows): One of the most popular media and video streaming platforms. They have over 8000 movies or tv shows available on their platform, as of mid-2021, they have over 200M Subscribers globally. This tabular dataset consists of listings of all the movies and tv shows available on Netflix, along with details such as - cast, directors, ratings, release year, duration, etc.

* [Amazon prime](https://www.kaggle.com/datasets/shivamb/amazon-prime-movies-and-tv-shows): Another one of the most popular media and video streaming platforms. They have close to 10000 movies or tv shows available on their platform, as of mid-2021, they have over 200M Subscribers globally. This tabular dataset consists of listings of all the movies and tv shows available on Amazon Prime, along with details such as - cast, directors, ratings, release year, duration, etc.*



In [77]:
import pandas as pd # queried_data processing
import pandas_profiling as pp
import numpy as np # linear algebra

In [78]:
df_netflix = pd.read_csv('Dataset/originalDataset/netflix_titles.csv')
df_amazon = pd.read_csv('Dataset/originalDataset/amazon_prime_titles.csv')
print(len(df_netflix), len(df_amazon))

8807 9668


# Prepering data

Objective is the one of concatenate amazon and netflix databases mantaing storage information. W
We add two colums: netflix and amazon both with value 1 or 0 representing the absence or presence of the title on the platform. To keep the date added information we rename columns to distinguish the relative streaming service.

In [79]:
df_netflix.drop(columns = df_netflix.columns[0], axis = 1, inplace= True)
df_netflix['netflix'] = 1
df_netflix['amazon'] = 0
df_netflix.rename(columns = {'date_added':'date_added_netflix'}, inplace = True)

df_amazon.drop(columns = df_amazon.columns[0], axis = 1, inplace= True)
df_amazon['amazon'] = 1
df_amazon['netflix'] = 0
df_amazon.rename(columns = {'date_added':'date_added_amazon'}, inplace = True)

In [80]:

dataset = pd.concat([df_netflix, df_amazon],axis=0, join="outer", sort=False)
dataset = dataset.reset_index(drop=True)
dataset.head(3)


,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1,0,NaN
1,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1,0,NaN
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,NaN


The concatenated dataset present some errors. Shows that are both present on Netflix and Amazon Prime are recoded twice in our dataset: first time having only netflix information, the second time only the amazon ones. We decide to extract unic triples from the original datasets containing year, type (movie or tv Series) and name. Then we check the triples that the two dataset has in common and we store them in a list named 'title'.

In [81]:
netflix = []
amazon = []

def union(df, new):
    for  i, x in df['title'].iteritems():
        year = df['release_year'][i]
        type = df['type'][i]
        movie = x
        new.append((year, type, movie))
    return new

union(df_netflix, netflix)
union(df_amazon, amazon)

print(len(netflix), len(amazon))

8807 9668


In [82]:
title = []
for (y,t,m) in netflix:
    if (y,t,m) in amazon:
        title.append((y,t,m))
len(title)

182

Finally, we iterate over the concatenated dataset querying only the shared titles merging amazon and Netflix information about possession and date of addition.
We decide to take description, country and cast information from the Netflix dataset because it was the best filled of the two. So, at the end of this process, we drop duplicates filtered by title, director, release year and type; keeping the first entries. Then we fill null value with 'No Data'.

In [83]:
df = dataset.copy()
df.replace(np.nan, 'null', inplace=True)
to_drop = set()
for i, r in df.iterrows(): 
    if (r['release_year'],r['type'],r['title']) in title:
        df.loc[i,'netflix'] = 1
        df.loc[i, 'amazon'] = 1
        q = df.query('title=="'+r['title']+'" & type=="'+r['type']+'" & release_year== '+ str(r['release_year']) +'')

        null_val = []      
        for j, x in q.iterrows():
            if x['date_added_netflix'] != 'null':
                df.loc[i, 'date_added_netflix'] = x['date_added_netflix']

            if x['date_added_amazon'] != 'null':
                df.loc[i, 'date_added_amazon'] = x['date_added_amazon']

            n = 0 
            for i in x.values:
                if i == 'null':
                    n+=1
            null_rate = n / len(x.values) * 100
            null_val.append((j,null_rate))
        
        if len(null_val) >= 2:
            row_to_keep = min(null_val, key=lambda tup: tup[1])
            null_val.remove(row_to_keep)
            for i, x in null_val:
                if x != 0.0:
                    to_drop.add(i)


df = df.drop(to_drop)

In [84]:
df.replace( 'null', np.nan, inplace=True)
for i in df.columns:
    null_rate = df[i].isna().sum() / len(df) * 100 
    if null_rate > 0 :
        print("{} null rate: {}%".format(i,round(null_rate,2)))

type null rate: 0.01%
title null rate: 0.01%
director null rate: 25.64%
cast null rate: 11.23%
country null rate: 52.86%
date_added_netflix null rate: 51.91%
release_year null rate: 0.01%
rating null rate: 1.83%
duration null rate: 0.02%
listed_in null rate: 0.01%
description null rate: 0.01%
netflix null rate: 0.01%
amazon null rate: 0.01%
date_added_amazon null rate: 99.16%


In [85]:
df['date_added_netflix'].replace(np.nan, 'No Data',inplace  = True)
df['date_added_amazon'].replace(np.nan, 'No Data',inplace  = True)
df['country'].replace(np.nan, 'No Data',inplace  = True)
df['director'].replace(np.nan, 'No Data',inplace  = True)
df['cast'].replace(np.nan, 'No Data',inplace  = True)
df['rating'].replace(np.nan, 'No Data',inplace  = True)
df['title'] = df['title'].replace({'"':''}, regex=True)
df['title'] = df['title'].replace({'\n':' '}, regex=True)
df = df.drop_duplicates(subset=['title','director', 'release_year', 'type'], keep='first')
df = df.dropna()
df = df.reset_index(drop=True)
df.to_csv('data.csv', index=False)

df.head(3)

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,"September 25, 2021",2020.0,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1.0,0.0,No Data
1,TV Show,Blood & Water,No Data,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021.0,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1.0,0.0,No Data
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021.0,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1.0,0.0,No Data


# Data enrichment 

Our data will be enriched using two sources:

* [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page): wikidata is the a free and open knowledge graph containing linked information used by the famouse online encyclopedia. 

* [iMdB](https://www.imdb.com/): Internet Movie Database is the world's most popular and authoritative source for movie, TV shows and celebrity content, where you can find ratings and reviews by creteque and public.

In [86]:
import pprint #indet json 
import requests #make http requests
import json
from qwikidata.sparql  import return_sparql_query_results #return sparql results
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl
from http.client import IncompleteRead
import time
import urllib.error
from textblob import Word

In order to query information, we split our dataset in Movies and TV Shows. Through wikidate we retrive missing information form our starting dataset, such like countries and directors. In addition, we add interesting information for us such as the gender of the director and the distributor. Finally, we also retrive the iMdb id of the movie for future query.

### Movies

In [87]:
movie_title = df.query("type == 'Movie'")
print(len(movie_title))

# movie_query = """
#             SELECT ?film_label ?director_label ?dir_gen_label ?distributor_label ?imdbID ?rottenscore
#             WHERE
#             {
#             ?film wdt:P31 wd:Q11424 .
#             ?film rdfs:label """+'"'+ x +'"' +"""@en .
#             ?film rdfs:label ?film_label .
#             FILTER(lang(?film_label) = 'en')
#             OPTIONAL {?film wdt:P57 ?director . 
#             ?director rdfs:label ?director_label .    
#             FILTER(lang(?director_label) = 'en')
#             OPTIONAL {?director wdt:P21 ?dir_gen . 
#             ?dir_gen rdfs:label ?dir_gen_label .
#             FILTER(lang(?dir_gen_label) = 'en')}}
#             OPTIONAL {?film wdt:P750 ?distributor . 
#             ?distributor rdfs:label ?distributor_label .
#             FILTER(lang(?distributor_label) = 'en')}
#             OPTIONAL {?film wdt:P345 ?imdbID.}
#             OPTIONAL {?film wdt:P444 ?rottenscore.}
#             }
#             """

13784


In [88]:
tv_series = df.query("type == 'TV Show'")
print(len(tv_series))

# tv_series_query = """
#             SELECT ?series_label ?director_label ?dir_gen_label ?distributor_label ?imdbID ?rottenscore
#             WHERE
#             {
#             ?series wdt:P31 wd:Q5398426 .
#             ?series rdfs:label """+'"'+ x +'"' +"""@en .
#             ?series rdfs:label ?series_label .
#             FILTER(lang(?series_label) = 'en')
#             OPTIONAL {?series wdt:P57 ?director . 
#             ?director rdfs:label ?director_label .    
#             FILTER(lang(?director_label) = 'en')
#             OPTIONAL {?director wdt:P21 ?dir_gen . 
#             ?dir_gen rdfs:label ?dir_gen_label .
#             FILTER(lang(?dir_gen_label) = 'en')}}
#             OPTIONAL {?series wdt:P449 ?distributor . 
#             ?distributor rdfs:label ?distributor_label .
#             FILTER(lang(?distributor_label) = 'en')}
#             OPTIONAL {?series wdt:P345 ?imdbID.}
#             OPTIONAL {?series wdt:P444 ?rottenscore.}
#             }
#             """

4505


In [89]:
# def wikidata(data, query):

#     title = []
#     director = []
#     gender = []
#     distributor = []
#     imdbID = []
#     rottenscore = []
#     not_found = []

    
#     # get the endpoint API
#     wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

#     for x in data():
        
#         try:
#             my_SPARQL_query = query
#             # set the endpoint 
#             sparql_wd = SPARQLWrapper(wikidata_endpoint)
#             # set the query
#             sparql_wd.setQuery(my_SPARQL_query)
#             # set the returned format
#             sparql_wd.setReturnFormat(JSON)
#             # get the results
            
#             results = sparql_wd.query().convert()

#             if results['results']['bindings'] == []:
#                 not_found.append(""+x+"")
                
#             else:
#                 title.append(results['results']['bindings'][0]['film_label']['value'])
#                 if "director_label" in results['results']['bindings'][0]:
#                     director.append(results['results']['bindings'][0]['director_label']['value'])
#                 else:
#                     director.append("no_data")
#                 if "dir_gen_label" in results['results']['bindings'][0]:
#                     gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
#                 else:
#                     gender.append("no_data")
#                 if "distributor_label" in results['results']['bindings'][0]:
#                     distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
#                 else:
#                     distributor.append("no_data")
#                 if "imdbID" in results['results']['bindings'][0]:
#                     imdbID.append(results['results']['bindings'][0]['imdbID']['value'])
#                 else:
#                     imdbID.append("no_data")
#                 if "rottenscore" in results['results']['bindings'][0]:
#                     rottenscore.append(results['results']['bindings'][0]['rottenscore']['value'])
#                 else:
#                     rottenscore.append("no_data")
                
#         except urllib.error.HTTPError as e:
#             time.sleep((int(e.headers["retry-after"])) + 1)
#             error_title = query.index(x)
#             wikidata(query[error_title:])

#     df = {"id": imdbID, "title": title, "director": director, "director_gender": gender, "distributor": distributor, "rating score": rottenscore}
#     return df
            
# movie_wiki = wikidata(movie_title['title'], movie_query)
# movie_wiki.to_csv('Dataset/query_results_data/wikidata_query_results/movie.csv', index = False)
# serie_wiki = wikidata(tv_series['title'], tv_series_query)
# serie_wiki.to_csv('Dataset/query_results_data/wikidata_query_results/TVShows.csvv', index = False)

In [90]:
movie_wiki = pd.read_csv('Dataset/query_results_data/wikidata_query_results/movie.csv')
serie_wiki = pd.read_csv('Dataset/query_results_data/wikidata_query_results/TVShows.csv')

print(len(movie_wiki), len(serie_wiki))

6666 1969


IMdb Api requests

In [91]:
# def imdb(data, filename):
#     json_list = []
#     for id in data:

#         url = "https://imdb-api.com/en/API/UserRatings/k_2xtxoo0v/"+id+""

#         resp = requests.get(url)
#         data = resp.json()
#         json_list.append(data)

#         with open(filename, 'w') as file_object:  #open the file in write mode
#             json.dump(json_list, file_object)

# imdb(movie_wiki['id'].to_list(), 'movie.json')
# imdb(serie_wiki['id'].to_list(), 'series.json')


In [92]:
f = open('Dataset/query_results_data/imdb_query_results/movie.json')
movie_imdb = json.load(f)

f = open('Dataset/query_results_data/imdb_query_results/series.json')
serie_imdb = json.load(f)

Uniamo i due risultati

In [93]:
def query_union(imdb, wiki):
    for x in range(len(imdb)):
        id = imdb[x]['id']
        if id:
            row = wiki.query('id == "'+id+'"')
            index = row.index.values
            if len(index) != 0:
                index = row.index.values[0]
                wiki.loc[index, 'title_imdb'] = imdb[x]['title']
                wiki.loc[index, 'release_date'] = imdb[x]['releaseDate']

                wiki.loc[index, 'directors'] = imdb[x]['directors']
                wiki.loc[index, 'cast'] = imdb[x]['stars']

                if wiki._get_value(index, 'distributor') == 'no_data':
                    wiki.loc[index, 'distributor'] = imdb[x]['companies']

                wiki.loc[index, 'countries'] = imdb[x]['countries']
                wiki.loc[index, 'languages'] = imdb[x]['languages']
                wiki.loc[index, 'rating'] = imdb[x]['contentRating']
                
                wiki.loc[index, 'imDbRating'] = imdb[x]['imDbRating']
                wiki.loc[index, 'imDbRatingVotes'] = imdb[x]['imDbRatingVotes']

                if wiki._get_value(index, 'rating score') == 'no_data':
                    if imdb[x]['ratings']:
                        wiki.loc[index, 'rating score'] = imdb[x]['ratings']['rottenTomatoes']

                if imdb[x]['boxOffice']:
                    wiki.loc[index, 'budget'] = imdb[x]['boxOffice']['budget']
                    wiki.loc[index, 'cumulativeWorldwideGross'] = imdb[x]['boxOffice']['cumulativeWorldwideGross']

                wiki.loc[index, 'keywords'] = imdb[x]['keywords']
                wiki.loc[index, 'awards'] = imdb[x]['awards']
    
    df = wiki
    return df

movie = query_union(movie_imdb, movie_wiki)
movie["release_date"] = pd.to_datetime(movie['release_date'])
movie['relese_year'] = movie['release_date'].dt.year

serie = query_union(serie_imdb, serie_wiki)
serie["release_date"] = pd.to_datetime(serie['release_date'])
serie['relese_year'] = serie['release_date'].dt.year

Aggiungiamo i dati a quelli di partenza, poi puliamo il dataset per ottenere il finale.

In [94]:
df = pd.read_csv('data.csv')
df

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Data,United States,"September 25, 2021",2020.0,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1.0,0.0,No Data
1,TV Show,Blood & Water,No Data,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021.0,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",1.0,0.0,No Data
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021.0,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1.0,0.0,No Data
3,TV Show,Jailbirds New Orleans,No Data,No Data,No Data,"September 24, 2021",2021.0,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",1.0,0.0,No Data
4,TV Show,Kota Factory,No Data,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021.0,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,1.0,0.0,No Data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18284,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",No Data,No Data,1940.0,7+,60 min,Comedy,New York City street principles get an East Si...,0.0,1.0,No Data
18285,TV Show,Planet Patrol,No Data,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",No Data,No Data,2018.0,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve...",0.0,1.0,No Data
18286,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",No Data,No Data,2008.0,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou...",0.0,1.0,No Data
18287,TV Show,Maradona: Blessed Dream,No Data,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",No Data,No Data,2021.0,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ...",0.0,1.0,No Data


In [95]:
def reconciliation(starting_data, queried_data, type):
    for i, r in starting_data.iterrows():
        results = queried_data['title'].to_list()
        if r['type'] == type:
            if r['title'] in results:
                index = results.index(r['title'])

                starting_data.loc[i, 'imdb_id'] = queried_data._get_value(index, 'id')
                starting_data.loc[i, 'languages'] = queried_data._get_value(index, 'languages')
                starting_data.loc[i, 'director_gender'] = queried_data._get_value(index, 'director gender')

                starting_data.loc[i, 'distributor'] = queried_data._get_value(index, 'distributor')
                starting_data.loc[i, 'imDbRating'] = queried_data._get_value(index, 'imDbRating')
                starting_data.loc[i, 'imDbRatingVotes'] = queried_data._get_value(index, 'imDbRatingVotes')
                starting_data.loc[i, 'rottenTomatoes'] = queried_data._get_value(index, 'rating score')

                starting_data.loc[i, 'budget'] = queried_data._get_value(index, 'budget')
                starting_data.loc[i, 'gross'] = queried_data._get_value(index, 'cumulativeWorldwideGross')
                starting_data.loc[i, 'keywords'] = queried_data._get_value(index, 'keywords')
                starting_data.loc[i, 'awards'] = queried_data._get_value(index, 'awards')
                
                if r['director'] == 'No Data':
                    starting_data.loc[i, 'director'] = queried_data._get_value(index, 'director')
                elif pd.isna(r['director']):
                    starting_data.loc[i, 'directors'] = queried_data._get_value(index, 'directors')

                if r['country'] == 'No Data':
                    starting_data.loc[i, 'country'] = queried_data._get_value(index, 'countries')
                if r['rating'] == 'No Data':
                    starting_data.loc[i, 'rating'] = queried_data._get_value(index, 'rating')
                if r['cast'] == 'No Data':
                    starting_data.loc[i, 'cast'] = queried_data._get_value(index, 'cast')

                if r['date_added_netflix'] == 'No Data' and r['netflix'] == 1.0 and queried_data._get_value(index, 'relese_year') > 2006.0:
                    starting_data.loc[i, 'date_added_netflix'] = queried_data._get_value(index, 'release_date')

                if r['date_added_amazon'] == 'No Data' and r['amazon'] == 1.0 and queried_data._get_value(index, 'relese_year') > 2006.0:
                    starting_data.loc[i, 'date_added_amazon'] = queried_data._get_value(index, 'release_date')

    final_dataset = starting_data
    return final_dataset

working = reconciliation(df, movie, 'Movie')
final = reconciliation(working, serie, 'TV Show')
final.replace( 'No Data', np.nan, inplace=True)
final.replace( 'None', np.nan, inplace=True)
final.replace( '', np.nan, inplace=True)
final.replace( 'no_data', np.nan, inplace=True)
final.dropna(subset=['imdb_id'], inplace=True)
final.reset_index(drop=True, inplace=True)
final = final[['type', 'imdb_id', 'title', 'director','director_gender', 'cast', 'distributor', 'country', 'languages','release_year', 'rating', 'duration', 'listed_in', 'netflix', 'amazon', 'date_added_netflix', 'date_added_amazon', 'imDbRating', 'imDbRatingVotes','rottenTomatoes', 'budget', 'gross', 'description', 'keywords', 'awards']]
final['distributor'] = final['distributor'].str.replace(r"['()']","")
final['duration'] = final['duration'].apply(lambda w: Word(w).singularize())


/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_17546/1466768670.py:52: FutureWarning: The default value of regex will change from True to False in a future version.
  final['distributor'] = final['distributor'].str.replace(r"['()']","")


In [96]:
for i in final.columns:
    null_rate = final[i].isna().sum() / len(final) * 100 
    if null_rate > 0 :
        print("{} null rate: {}%".format(i,round(null_rate,2)))

director null rate: 15.92%
director_gender null rate: 27.59%
cast null rate: 0.23%
distributor null rate: 23.03%
country null rate: 13.75%
languages null rate: 24.06%
rating null rate: 0.01%
date_added_netflix null rate: 39.87%
date_added_amazon null rate: 84.16%
imDbRating null rate: 22.87%
imDbRatingVotes null rate: 22.87%
rottenTomatoes null rate: 53.32%
budget null rate: 78.98%
gross null rate: 68.44%
keywords null rate: 27.62%
awards null rate: 50.54%


In [97]:
final_dataset = final.copy()

### Director Gender - 27% null
We compile missing director gender values using [genderize](https://pypi.org/project/Genderize/) for py library. It convert any english name into its gender. 

In [98]:
import gender_guesser.detector as gender
d = gender.Detector()
missing_gender = final_dataset.query('director.notnull() & director_gender.isnull()')

In [99]:
andy = 0
for  index, name in missing_gender['director'].iteritems(): 
    if not pd.isna(name):
        full_names = name.split(', ')
        genders = []
        for i in full_names:
            name = i.split()[0]
            if '.' in name:
                name = i.split()[1]
            gender = d.get_gender(name)
            if gender == 'mostly_male':
                gender = 'male'
            elif gender == 'mostly_female':
                gender = 'female'
            elif gender == 'andy':
                if (andy % 2) == 0:
                    gender = 'male'
                else:
                    gender = 'female'
                andy += 1
            genders.append(gender)
        final_dataset.loc[index, 'director_gender'] = ', '.join(genders)

### Languages - 22.76% null
We compile missing country values using converting languages into its country. 

In [100]:
from countryinfo import CountryInfo
from babel import Locale
missing_languages = final_dataset.query('country.notnull() & languages.isnull()')

LANG_MAPPING = {
    'Soviet Union' : 'Russian',
    'Bahamas': 'English',
    'Kosovo': 'Albanian, Serbian',
    'South Africa': 'English',
    'United Kingdom': 'English',
    'gn': 'Guarani',
    'nr': 'South Ndebele',
    'st': 'Sotho',
    'tn': 'Tswana',
    'ts' : 'Tsonga',
    've' : 'Venda',
}

In [101]:
for index, x in missing_languages['country'].iteritems():
    if not pd.isna(x):
        to_add = set()
        l = x.split(', ')
        for i in range(len(l)):
            name = l[i].replace(',', '')
            if name in LANG_MAPPING.keys():
                lang = LANG_MAPPING[name]
                to_add.add(lang)
            else:
                country = CountryInfo(name)
                data = country.languages()
                list = set()
                for item in data:
                    if item in LANG_MAPPING.keys():
                        lang = LANG_MAPPING[item]
                        list.add(lang)
                    else:
                        language = Locale.parse(item)
                        language = language.get_language_name('en')
                        list.add(language)
                to_add.add(', '.join(list)) 

        final_dataset.loc[index, 'languages'] = ', '.join(to_add)

### Rating - normalization
We compile missing country values using converting languages into its country.

In [102]:
for index, rating in final_dataset['rating'].iteritems():
    if not pd.isna(rating):
        if rating == 'ALL'or rating == 'TV-G'or rating == 'TV-Y'or rating == 'G':
            final_dataset.loc[index, 'rating']  = 'Kids All'
        elif rating == '7+'or rating == 'PG'or rating == 'TV-PG'or rating == 'TV-Y7'or rating == 'TV-Y7-FV' :
            final_dataset.loc[index, 'rating']  = 'Older Kids 7+'
        elif rating == '13+'or rating == 'PG-13' :
            final_dataset.loc[index, 'rating']  = 'Teens 13+'
        elif rating == '16'or rating == '16+'or rating == 'AGES_16_'or rating == 'TV-14':
            final_dataset.loc[index, 'rating']  = 'Young Adults 16+'
        elif rating == '18+'or rating == 'AGES_18_'or rating == 'TV-MA'or rating == 'NC-17' or rating == 'R':
            final_dataset.loc[index, 'rating']  = 'Adults 18+'
        elif rating == 'NR'or rating == 'UNRATED'or rating == 'UR'or rating == 'NOT_RATE' or rating == 'TV-NR':
            final_dataset.loc[index, 'rating']  = 'Unrated'

### Listed in - normalization

In [103]:
GEN_MAPPING = {
    'Lgbtq movie': 'Lgbtq',
    'Suspense':'Thriller',
     'Sports movie':'Sport',
     'Anime feature':'Anime',
     'Anime series':'Anime',
    'International movie':'International',
    'International tv show':'International',
     'Science fiction': 'Sci-fi & fantasy',
      'Tv sci-fi & fantasy':'Sci-fi & fantasy',
    'Fantasy':'Sci-fi & fantasy',
     'Tv drama' : 'Drama',
    'Horror movie':'Horror',
    'Tv horror' :'Horror',
    'Adventure':'Action & adventure',
    'Action':'Action & adventure',
    'Tv action & adventure':'Action & adventure',
    'Romantic movie':'Romance',
    'Faith and spirituality':'Faith & spirituality',
     'Arthouse': 'Art',
       'And culture': 'Art',
    'Movie':'Cult movie',
    'Special interest':'Independent movie', 
    'Kid':'Children & family movie',
     'Young adult audience':'Children & family movie',
     'Stand-up comedy': 'Stand-up comedy & talk show',
     'Tv thriller':'Thriller',
     'Tv comedy': 'Commedy'

    }

In [104]:
df = pd.DataFrame()

df['genre'] = final_dataset['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 
for index, i in df['genre'].iteritems(): 
    for n in range(len(i)):
        w = i[n]
        w = Word(w).singularize().capitalize()
        if w in GEN_MAPPING.keys():       
            w = GEN_MAPPING[w]
        i = i[:n]+[w]+i[n+1:]
    final_dataset.loc[index, 'listed_in'] = ', '.join(i)


### rottenTomatoes normalization

In [105]:
for  index, vote in final_dataset['rottenTomatoes'].iteritems():
    if not pd.isna(vote) and type(vote) != float:
        l = (vote.split())
        if len(l) > 1:
            l.remove('of')
            val = float(l[0])*100//float(l[1])
        if '%' in vote:
            val = float(vote.replace('%', ''))
        elif '/' in vote:
            l = vote.split('/')
            val = float(l[0]) * 100 // float(l[1])
        else:
            val = float(l[0])
            
        final_dataset.loc[index, 'rottenTomatoes'] = val

### Imdb normalization

In [106]:
for  index, imbdvote in final['imDbRating'].iteritems():
    if pd.notnull(imbdvote):
        val = float(imbdvote)*10
        final_dataset.loc[index, 'imDbRating'] = val

### awards normalization

In [107]:
for  index, x in final_dataset['awards'].iteritems():
    if not pd.isna(x) and x != '':
        list = x.split(", ")
        
        k = list[0]

        special = k.split('|')

        nameA =[]
        tot = []
        stat= []
        for item in special:     
            l = item.split()
            
            if 'for' in l:
                l.remove('for')

            if 'Top' not in l and len(l) > 1:
                stat.append(l[0])
                tot.append(int(l[1]))
                name = ' '.join(l[2:])
                name = name.removesuffix('s')
                nameA.append(name)
                
            elif 'Top' in l and len(l) > 1:
                tot.append(1)
                name = ' '.join(l[:3])
                nameA.append(name)
                stat.append('nn')
                
            else:
                if l[0] != 'Nominated' and l[0] != 'Won':
                    nameA.append(l[0])
                    tot.append(1)
                    stat.append('nn')

        stat = ', '.join(stat)
        nameA = ', '.join(nameA)
        final_dataset.loc[index, 'special_award_name']  = nameA
        if len(tot) > 1:
            final_dataset.loc[index, 'special_award_tot']  = str(tot)
        else:
            final_dataset.loc[index, 'special_award_tot']  = tot[0]
        final_dataset.loc[index, 'special_award_stat']  = stat

        w = list[1].split('&')
        win_tot = []
        nomination_tot = []
        
        for n in range(len(w)):
            if 'win' in  w[n]  and 'nominations' not in w[n]:
                nom = w[n].split()
                win_tot.append(int(nom[0]))
            if 'nomination' in w[n] and 'win' not in w[n]:
                win = w[n].split()
                nomination_tot.append(int(win[0]))
        
        if nomination_tot != []:
            final_dataset.loc[index, 'award_nomination_tot']  = nomination_tot[0]
        else :
            final_dataset.loc[index, 'award_nomination_tot']  = np.nan

        if win_tot != []:
            final_dataset.loc[index, 'award_win_tot']  = win_tot[0]
        else :
            final_dataset.loc[index, 'award_win_tot']  = np.nan
 
    else:
        final_dataset.loc[index, 'special_award_name']  = np.nan        
        final_dataset.loc[index, 'special_award_tot']  = np.nan        
        final_dataset.loc[index, 'special_award_stat']  = np.nan        
        final_dataset.loc[index, 'award_win_tot']  = np.nan        
        final_dataset.loc[index, 'award_nomination_tot']  = np.nan
        

### budget normalization

In [108]:
import re
from currency_converter import CurrencyConverter
c = CurrencyConverter()

CURRENCY_MAPPING = {
    '$' : 'USD',
    'A$': 'AUD',
    'CA$': 'CAD',
    'CN¥': 'CNY',
    'HK$': 'HKD',
    'MX$': 'MXN',
    'NT$': 'TWD',
    'RUR': 'RUB',
    '£' : 'GBP',
    '₩' : 'KRW',
    '₪' : 'ILS',
    '€' : 'EUR',
    '₹' : 'INR',
}

CURRENCY_EXRATE = {
    'ARS' : 0.0070, 
    'EGP' : 0.052, 
    'MVR' : 0.065, 
    'NGN' : 0.0023, 
    'PKR' : 0.0043, 
    'TWD' : 0.032,
}

In [109]:
for index, x in final_dataset['budget'].iteritems():
    if not pd.isna(x):
        x = x.split()
        if '(estimated)' in x:
            x.remove('(estimated)')
        if len(x) == 1:
            val = x[0].split(',')
            val = ''.join(val)
            currency = [s for s in re.findall(r"[^0-9.]", val)]
            currency = ''.join(currency)
            tot = val.strip(currency)
        elif len(x) != 0:
            tot = x[1].split(',')
            tot = ''.join(tot)
            currency = x[0]

        if currency in CURRENCY_MAPPING.keys():
            currency = CURRENCY_MAPPING[currency]

        if currency in c.currencies:
            ammount = int(c.convert(tot, currency))
        else:
            ex_Rate = CURRENCY_EXRATE[currency]
            ammount = int(int(tot) // ex_Rate)

        final_dataset.loc[index, 'budget']  = ammount 
    else:
        final_dataset.loc[index, 'budget']  = np.nan

### gross normalization

In [110]:
c = CurrencyConverter()
for index, x in final_dataset['gross'].iteritems():
    if not pd.isna(x):
        x = x.split(',')
        x = ''.join(x)
        x = [int(s) for s in re.findall(r'\b\d+\b', x)]
        ammount = int(c.convert(x[0], 'USD'))
        final_dataset.loc[index, 'gross']  =  ammount
    else:
        final_dataset.loc[index, 'gross']  = np.nan

### keywords 26.47%

In [111]:
import spacy
keywords_to_find = final_dataset.query('keywords.isnull() and description.notnull()')
NER = spacy.load("en_core_web_sm")
to_take = ['NOUN', 'PROPN']
keywords = []
for index, x in keywords_to_find['description'].iteritems():
    parsed = NER(x)
    tokens = []
    for token in parsed:
        if token.pos_ in to_take:
            if token.text not in tokens:
                tokens.append(token.text)
    keys = ','.join(tokens)
    final_dataset.loc[index, 'keywords']  = keys

### Final Dataset

In [112]:
final_dataset.replace( np.nan, 'no_data', inplace=True)
final_dataset.to_csv('final_dataset.csv', index= False)

# data analysis

In [113]:
final_dataset.query('director_gender == "andy"')

,type,imdb_id,title,director,director_gender,cast,distributor,country,languages,release_year,...,budget,gross,description,keywords,awards,special_award_name,special_award_tot,special_award_stat,award_nomination_tot,award_win_tot


# data visualization